# Problem Modeling With `qiskit-addon-opt-mapper`

In [1]:
# Uncomment and run this cell to install requirements for the notebook
# IMPORTANT: cplex requires Python 3.10. You can choose not to install it and not run the cells that require cplex.

# !pip install cplex
# !pip install qiskit-addon-opt-mapper==0.1.0

This notebook contains practical examples of how to apply the `qiskit-addon-opt-mapper` for the different steps of the QAOA modeling workflow.

For more in-depth tutorials, check out the qiskit-addon-opt-mapper how-to section.

<img src="images/qaoa_workflow.png" width="1200">

## 1. Model the optimization problem

This step consists on defining an `OptimizationProblem` instance and adding relevant variables, objective function and constraints.

### 1.1. Direct Modeling

In [2]:
from qiskit_addon_opt_mapper import OptimizationProblem

op1 = OptimizationProblem("example_problem_1")

op1.binary_var(name="b")
op1.integer_var(name="i", lowerbound=-1, upperbound=5)
op1.continuous_var(name="c", lowerbound=-1, upperbound=5)
op1.spin_var(name="s")

op1.minimize(
    constant=3,
    linear={"b": 1},
    quadratic={("b", "i"): 2, ("c", "c"): -1},
    higher_order={
        3: {("b", "i", "c"): 4},
        4: {("b", "s", "c", "i"): -2},
    },
)

op1.linear_constraint(
    linear={"b": 1, "i": 2}, sense="==", rhs=3, name="lin_eq"
)

<LinearConstraint: b + 2*i == 3 'lin_eq'>

In [3]:
op1.quadratic_constraint(
    linear={"b": 1, "i": 1},
    quadratic={("b", "b"): 1, ("i", "c"): -1},
    sense="==",
    rhs=1,
    name="quad_eq",
)
op1.higher_order_constraint(
    linear={"b": 1},
    quadratic={("b", "b"): 1, ("i", "c"): -1},
    higher_order={
        3: {("b", "i", "c"): 1},
        4: {("b", "i", "c", "s"): 2},
    },
    sense=">=",
    rhs=1,
    name="hoc_geq",
)
print(op1.prettyprint())

Problem name: example_problem_1

Minimize
  -2*b*c*i*s + 4*b*c*i + 2*b*i - c^2 + b + 3

Subject to
  Linear constraints (1)
    b + 2*i == 3  'lin_eq'

  Quadratic constraints (1)
    b^2 - i*c + b + i == 1  'quad_eq'

  Higher-order constraints (1)
    2*b*c*i*s + b*c*i + b^2 - i*c + b >= 1  'hoc_geq'

  Integer variables (1)
    -1 <= i <= 5

  Continuous variables (1)
    -1 <= c <= 5

  Binary variables (1)
    b

  Spin variables (1)
    -1 <= s <= 1



### 1.2. From docplex

In [4]:
from docplex.mp.model import Model

docplex_mod = Model(name="docplex model")
x = docplex_mod.binary_var(name="x")
y = docplex_mod.integer_var(name="y", lb=-1, ub=5)
z = docplex_mod.continuous_var(name="z", lb=-1, ub=5)

docplex_mod.minimize(3 + x + 2 * x * y - z * z)
docplex_mod.add_constraint(x + 2 * y == 3, "lin_eq")
docplex_mod.add_constraint(x + 2 * y <= 3, "lin_leq")
docplex_mod.add_constraint(x + 2 * y >= 3, "lin_geq")

from qiskit_addon_opt_mapper.translators import from_docplex_mp

op2 = from_docplex_mp(docplex_mod)
print(op2.prettyprint())

Problem name: docplex model

Minimize
  2*x*y - z^2 + x + 3

Subject to
  Linear constraints (3)
    x + 2*y == 3  'lin_eq'
    x + 2*y <= 3  'lin_leq'
    x + 2*y >= 3  'lin_geq'

  Integer variables (1)
    -1 <= y <= 5

  Continuous variables (1)
    -1 <= z <= 5

  Binary variables (1)
    x



### 1.3. From LP File

In [5]:
# WARNING: This cell requires cplex. Skip if cplex not installed.

from docplex.mp.model_reader import ModelReader

model = ModelReader.read("data/my_lp_file.lp")

from qiskit_addon_opt_mapper.translators import from_docplex_mp

op3 = from_docplex_mp(model)

In [6]:
print(op3.prettyprint())

Problem name: my_lp_file

Minimize
  2*x0^2 + 0.5*x0*x1 + 0.5*x1^2 + 2*x0 + 2*x1

Subject to
  Linear constraints (1)
    x0 + x1 == 2  'constraint'

  Integer variables (2)
    0 <= x0 <= 4
    0 <= x1 <= 4



### 1.4. From Application

In [7]:
import rustworkx as rx
from qiskit_addon_opt_mapper.applications import IndependentSet

edges = [
    (0, 1, 1),
    (0, 4, 1),
    (1, 2, 1),
    (1, 4, 1),
    (2, 3, 1),
    (3, 4, 1),
]
graph = rx.PyGraph()
graph.add_nodes_from(range(5))
graph.add_edges_from(edges)

# instantiate application class
mis = IndependentSet(graph)

# convert to optimization problem
op4 = mis.to_optimization_problem()
print(op4.prettyprint())

Problem name: Independent set

Maximize
  x_0 + x_1 + x_2 + x_3 + x_4

Subject to
  Linear constraints (6)
    x_0 + x_1 <= 1  'c0'
    x_0 + x_4 <= 1  'c1'
    x_1 + x_2 <= 1  'c2'
    x_1 + x_4 <= 1  'c3'
    x_2 + x_3 <= 1  'c4'
    x_3 + x_4 <= 1  'c5'

  Binary variables (5)
    x_0 x_1 x_2 x_3 x_4



## 1.2.-1.4. Convert between formulations

Steps 2 to 4 of the workflow cover different formulation conversions for variables, constraints and penalty terms. The addon offers both the individual conversion tools and end-to-end conversions to the most common formulations: QUBO and HUBO. 

<img src="images/convert_formulations.png" width="1200">

### A. Convert to QUBO step by step

In [8]:
from qiskit_addon_opt_mapper.converters import *

op6 = OptimizationProblem("Quadratic Program to QUBO Example")
op6.integer_var(name="x", lowerbound=0, upperbound=3)
op6.binary_var(name="y")
op6.maximize(linear={"x": 2}, quadratic={("x", "y"): 1})
op6.linear_constraint(
    linear={"x": 1, "y": 1}, sense="LE", rhs=5, name="c1"
)

# Step 1: Convert linear inequalities to penalties
ineq_to_penalty = LinearInequalityToPenalty(penalty=1e5)
op6_converted = ineq_to_penalty.convert(op6)
# Step 2: Convert inequalities to equalities
op6_converted = InequalityToEquality().convert(op6_converted)
# Step 3: Convert integer variables to binary
op6_converted = IntegerToBinary().convert(op6_converted)
# Step 4: Convert spin variables to binary
op6_converted = SpinToBinary().convert(op6_converted)
# Step 5: Convert equalities to penalties
eq_to_penalty = EqualityToPenalty(penalty=1e5)
op6_converted = eq_to_penalty.convert(op6_converted)
# Step 6: Convert maximization to minimization
op6_converted = MaximizeToMinimize().convert(op6_converted)

In [9]:
print(op6_converted.prettyprint())
print(
    f"Penalty used: {LinearInequalityToPenalty(penalty=1e5).penalty}"
)

Problem name: 

Minimize
  100000*c1@int_slack@0^2 + 400000*c1@int_slack@0*c1@int_slack@1
  + 400000*c1@int_slack@0*c1@int_slack@2 + 400000*c1@int_slack@1^2
  + 800000*c1@int_slack@1*c1@int_slack@2 + 400000*c1@int_slack@2^2
  + 200000*x@0*c1@int_slack@0 + 400000*x@0*c1@int_slack@1
  + 400000*x@0*c1@int_slack@2 + 100000*x@0^2 + 400000*x@0*x@1 + 199999*x@0*y
  + 400000*x@1*c1@int_slack@0 + 800000*x@1*c1@int_slack@1
  + 800000*x@1*c1@int_slack@2 + 400000*x@1^2 + 399998*x@1*y
  + 200000*y*c1@int_slack@0 + 400000*y*c1@int_slack@1 + 400000*y*c1@int_slack@2
  + 100000*y^2 - 1000000*c1@int_slack@0 - 2000000*c1@int_slack@1
  - 2000000*c1@int_slack@2 - 1000002*x@0 - 2000004*x@1 - 1000000*y + 2500000

Subject to
  No constraints

  Binary variables (6)
    x@0 x@1 y c1@int_slack@0 c1@int_slack@1 c1@int_slack@2

Penalty used: 100000.0


### B. Convert to QUBO directly

In [10]:
from qiskit_addon_opt_mapper.converters import (
    OptimizationProblemToQubo,
)

op6 = OptimizationProblem("Quadratic Program to QUBO Example")
op6.integer_var(name="x", lowerbound=0, upperbound=3)
op6.binary_var(name="y")
op6.maximize(linear={"x": 2}, quadratic={("x", "y"): 1})
op6.linear_constraint(
    linear={"x": 1, "y": 1}, sense="LE", rhs=5, name="c1"
)

# Convert the optimization problem to QUBO format
qubo_converter = OptimizationProblemToQubo(penalty=1e5)
op6_qubo = qubo_converter.convert(op6)

In [11]:
print("Original Problem:")
print(op6.prettyprint())

print("\nConverted QUBO Problem:")
print(op6_qubo.prettyprint())
print(f"penalty: {qubo_converter.penalty}")

Original Problem:
Problem name: Quadratic Program to QUBO Example

Maximize
  x*y + 2*x

Subject to
  Linear constraints (1)
    x + y <= 5  'c1'

  Integer variables (1)
    0 <= x <= 3

  Binary variables (1)
    y


Converted QUBO Problem:
Problem name: 

Minimize
  100000*c1@int_slack@0^2 + 400000*c1@int_slack@0*c1@int_slack@1
  + 400000*c1@int_slack@0*c1@int_slack@2 + 400000*c1@int_slack@1^2
  + 800000*c1@int_slack@1*c1@int_slack@2 + 400000*c1@int_slack@2^2
  + 200000*x@0*c1@int_slack@0 + 400000*x@0*c1@int_slack@1
  + 400000*x@0*c1@int_slack@2 + 100000*x@0^2 + 400000*x@0*x@1 + 199999*x@0*y
  + 400000*x@1*c1@int_slack@0 + 800000*x@1*c1@int_slack@1
  + 800000*x@1*c1@int_slack@2 + 400000*x@1^2 + 399998*x@1*y
  + 200000*y*c1@int_slack@0 + 400000*y*c1@int_slack@1 + 400000*y*c1@int_slack@2
  + 100000*y^2 - 1000000*c1@int_slack@0 - 2000000*c1@int_slack@1
  - 2000000*c1@int_slack@2 - 1000002*x@0 - 2000004*x@1 - 1000000*y + 2500000

Subject to
  No constraints

  Binary variables (6)
    x

## 1.5. Translate to Ising

The final step of the workflow is the translation to Ising, which is performed by a convenience function applied to the desired `OptimizationProblem` formulation. In the example below, we model a HUBO and apply `to_ising`.

In [12]:
from qiskit_addon_opt_mapper.converters import (
    OptimizationProblemToHubo,
)
from qiskit_addon_opt_mapper.translators import to_ising

op7 = OptimizationProblem("Optimization Problem to HUBO Example")
op7.integer_var(name="x", lowerbound=0, upperbound=3)
op7.binary_var(name="y")
op7.binary_var(name="z")
op7.minimize(
    linear={"x": 2},
    quadratic={("x", "y"): 1},
    higher_order={3: {("x", "y", "z"): 1}},
)
op7.higher_order_constraint(
    higher_order={3: {("x", "y", "z"): 1}},
    sense="EQ",
    rhs=2,
    name="c1",
)
hubo_converter = OptimizationProblemToHubo(penalty=1e5)
op7_hubo = hubo_converter.convert(op7)

ising_hubo, offset = to_ising(op7_hubo)

In [13]:
print(ising_hubo)

SparsePauliOp(['IIIZ', 'IIZI', 'IZIZ', 'IZII', 'IZZI', 'ZIII', 'ZIIZ', 'ZZII', 'ZZIZ', 'ZIZI', 'ZZZI', 'IIII', 'IIZZ', 'IZZZ', 'ZIZZ', 'ZZZZ'],
              coeffs=[ 12498.625+0.j,  24997.25 +0.j, -12499.625+0.j,  62498.875+0.j,
 -24999.25 +0.j,  62499.625+0.j, -12499.875+0.j, -62499.625+0.j,
  12499.875+0.j, -24999.75 +0.j,  24999.75 +0.j,  73437.5  +0.j,
  25000.   +0.j, -25000.   +0.j, -25000.   +0.j,  25000.   +0.j])


## BONUS: Model Validation using Solvers

While not always part of the workflow, this example showcases how solvers can be used for model validation.

In [14]:
# WARNING: This cell requires cplex. Skip if cplex not installed.

from qiskit_addon_opt_mapper.applications import IndependentSet

mis = IndependentSet(graph)
op8 = mis.to_optimization_problem()

from qiskit_addon_opt_mapper.converters import (
    LinearInequalityToPenalty,
)

penalty = 0.5
converter = LinearInequalityToPenalty(penalty)
converted_op = converter.convert(op8)

from qiskit_addon_opt_mapper.solvers import CplexSolver

validation_solver = CplexSolver()
result = validation_solver.solve(converted_op)
y = converter.interpret(result.x)
op8.is_feasible(y)

penalty2 = 2
converter2 = LinearInequalityToPenalty(penalty=penalty2)
converted_op2 = converter2.convert(op8)
result2 = validation_solver.solve(converted_op2)
y2 = converter.interpret(result2.x)
op8.is_feasible(y2)

True

In [15]:
print(y)
print(y2)

[1. 1. 1. 1. 0.]
[1. 0. 1. 0. 0.]
